input example:  
請輸入起始站:高雄  
請輸入終點站:屏東  
請輸入查詢日期(格式:10/10):10/18  
請輸入出發起始時間(格式:09:00):09:00  
請輸入出發結束時間(格式:12:00):12:00  

<pre>
step1:使用requests到台鐵網站抓取資料  
      step1-1:輸入資料  
      step1-2:post_data參數方式傳入  
      step1-3:發出requests.post  
step2:使用BeautifulSoup剖析資料  
      step2-1:剖析表格,並使用迴圈一列一列抓出來  
      step2-2:剖析一列中的資料  
      step2-3:印出資料  

1.requests  
2.BeautifulSoup  
3.post_data參數方式傳入  
4.每個TR格式如下:  
</pre>



```
<tr class="trip-column">
<td>
<ul class="train-number">
<li>
<span class="icon-fa icon-train"></span>
<a class="links" href="/tra-tip-web/tip/tip001/tip112/querybytrainno?rideDate=2020/10/18&amp;trainNo=3111" target="_blank" title="列車時刻表(另開新視窗)">區間3111</a>
<span>(</span>
<span class="location">臺南</span>
<span>→</span>
<span class="location">潮州</span>
<span>)</span>
</li>
</ul>
</td>
<td>05:24</td>
<td>06:21</td>
<td>57 分</td>
<td>-</td>
<td class="more">
<!-- 								<button type="button" class="icon chevron-down toggleBtn" title="展開收合表格" th:title="#{tip.tip115.foldTable}" onclick="tipDivToggleHideShow(this, 'chevron-up', 'chevron-down')">詳細</button> -->
<button class="icon chevron-down toggleTableBtn" title="展開收合表格" type="button">詳細</button>
</td>
<!-- 								<td><img th:each="equip:${timeTable.trainEquipList}" th:src="@{${'/images/' + equip.imgId + '.png'}}" th:title="${equip.name}" th:alt="${equip.name}"> -->
<!-- 									<div class="note" th:text="${timeTable.comment}">莒光號自由行，不發售團體票及無座票。另不提供TR-PASS學生版搭乘。</div></td> -->
<td>$ 68</td>
<td>$ 34</td>
<td class="check-way">
</td>
</tr>
```



In [2]:

#Post
import requests
from bs4 import BeautifulSoup
stations={'基隆': '0900', '七堵': '0930', '南港': '0980', '松山': '0990', '臺北': '1000', '萬華': '1010', '板橋': '1020', '樹林': '1040', '桃園': '1080', '中壢': '1100', '新竹': '1210', '竹南': '1250', '苗栗': '3160', '豐原': '3230', '臺中': '3300', '彰化': '3360', '員林': '3390', '斗六': '3470', '嘉義': '4080', '新營': '4120', '臺南': '4220', '岡山': '4310', '新左營': '4340', '高雄': '4400', '屏東': '5000', '潮州': '5050', '臺東': '6000', '玉里': '6110', '花蓮': '7000', '蘇澳新': '7130', '宜蘭': '7190', '瑞芳': '7360', '五堵': '0950', '汐止': '0960', '汐科': '0970', '浮洲': '1030', '南樹林': '1050', '山佳': '1060', '鶯歌': '1070', '福隆': '7290', '貢寮': '7300', '雙溪': '7310', '牡丹': '7320', '三貂嶺': '7330', '大華': '7331', '十分': '7332', '望古': '7333', '嶺腳': '7334', '平溪': '7335', '菁桐': '7336', '猴硐': '7350', '八斗子': '7362', '四腳亭': '7380', '日南': '2190', '大甲': '2200', '臺中港': '2210', '清水': '2220', '沙鹿': '2230', '龍井': '2240', '大肚': '2250', '追分': '2260', '泰安': '3210', '后里': '3220', '栗林': '3240', '潭子': '3250', '頭家厝': '3260', '松竹': '3270', '太原': '3280', '精武': '3290', '五權': '3310', '大慶': '3320', '烏日': '3330', '新烏日': '3340', '成功': '3350', '大湖': '4290', '路竹': '4300', '橋頭': '4320', '楠梓': '4330', '左營': '4350', '內惟': '4360', '美術館': '4370', '鼓山': '4380', '三塊厝': '4390', '民族': '4410', '科工館': '4420', '正義': '4430', '鳳山': '4440', '後庄': '4450', '九曲堂': '4460', '嘉北': '4070', '濁水': '3432', '龍泉': '3433', '集集': '3434', '水里': '3435', '車埕': '3436', '花壇': '3370', '大村': '3380', '永靖': '3400', '社頭': '3410', '田中': '3420', '二水': '3430', '源泉': '3431', '北新竹': '1190', '千甲': '1191', '新莊': '1192', '三姓橋': '1220', '香山': '1230', '林內': '3450', '石榴': '3460', '斗南': '3480', '石龜': '3490', '大林': '4050', '民雄': '4060', '水上': '4090', '南靖': '4100', '漢本': '7070', '武塔': '7080', '南澳': '7090', '東澳': '7100', '永樂': '7110', '蘇澳': '7120', '新馬': '7140', '冬山': '7150', '羅東': '7160', '中里': '7170', '二結': '7180', '四城': '7200', '礁溪': '7210', '頂埔': '7220', '頭城': '7230', '外澳': '7240', '龜山': '7250', '大溪': '7260', '大里': '7270', '石城': '7280', '六塊厝': '4470', '歸來': '5010', '麟洛': '5020', '西勢': '5030', '竹田': '5040', '崁頂': '5060', '南州': '5070', '鎮安': '5080', '林邊': '5090', '佳冬': '5100', '東海': '5110', '枋寮': '5120', '加祿': '5130', '內獅': '5140', '枋山': '5160', '富里': '6080', '東竹': '6090', '東里': '6100', '三民': '6120', '瑞穗': '6130', '富源': '6140', '大富': '6150', '光復': '6160', '萬榮': '6170', '鳳林': '6180', '南平': '6190', '林榮新光': '6200', '豐田': '6210', '壽豐': '6220', '平和': '6230', '志學': '6240', '吉安': '6250', '北埔': '7010', '景美': '7020', '新城': '7030', '崇德': '7040', '和仁': '7050', '和平': '7060', '北湖': '1150', '湖口': '1160', '新豐': '1170', '竹北': '1180', '竹中': '1193', '六家': '1194', '上員': '1201', '榮華': '1202', '竹東': '1203', '橫山': '1204', '九讚頭': '1205', '合興': '1206', '富貴': '1207', '內灣': '1208', '大武': '5190', '瀧溪': '5200', '金崙': '5210', '太麻里': '5220', '知本': '5230', '康樂': '5240', '山里': '6010', '鹿野': '6020', '瑞源': '6030', '瑞和': '6040', '關山': '6050', '海端': '6060', '池上': '6070', '後壁': '4110', '柳營': '4130', '林鳳營': '4140', '隆田': '4150', '拔林': '4160', '善化': '4170', '南科': '4180', '新市': '4190', '永康': '4200', '大橋': '4210', '保安': '4250', '仁德': '4260', '中洲': '4270', '長榮大學': '4271', '沙崙': '4272', '內壢': '1090', '埔心': '1110', '楊梅': '1120', '富岡': '1130', '新富': '1140', '三坑': '0910', '八堵': '0920', '百福': '0940', '海科館': '7361', '暖暖': '7390', '崎頂': '1240', '談文': '2110', '大山': '2120', '後龍': '2130', '龍港': '2140', '白沙屯': '2150', '新埔': '2160', '通霄': '2170', '苑裡': '2180', '造橋': '3140', '豐富': '3150', '南勢': '3170', '銅鑼': '3180', '三義': '3190', '潮州基地': '5999', '中興': '6022', '初鹿': '6023', '檳榔': '6025', '馬蘭': '6027', '舊臺東': '6028', '月美': '6045', '德高': '6055', '三臺': '6073', '富南': '6077', '富北': '6085', '萬寧': '6095', '安通': '6103', '樂合': '6107', '大禹': '6115', '瑞北': '6135', '大興': '6155', '干城': '6245', '田浦': '6251', '舊花蓮': '6252', '民立': '6253', '新村': '6254', '美崙': '6255', '深澳': '7363', '華山': '0995', '舊豐富': '3155', '林森': '4230', '南臺南': '4240', '建興': '5115', '香蘭': '5215', '三和': '5225'}
sstation=input("請輸入起始站:")
estation=input("請輸入終點站:")
sdate=input("請輸入查詢日期(格式:10/31):")
stime=input("請輸入出發起始時間(格式:09:00):")
etime=input("請輸入出發結束時間(格式:12:00):")

url="https://www.railway.gov.tw/tra-tip-web/tip/tip001/tip112/querybytime"
post_data={"startStation": stations[sstation]+"-"+sstation,"endStation": stations[estation]+"-"+estation,"transfer":"ONE","rideDate":"2020/"+sdate,"startOrEndTime":"true","startTime":stime,"endTime":etime,"trainTypeList":"ALL","_csrf":"f021ebab-6bac-4a5b-b720-27ea7d3569cd","query":"查詢"}
#print(post_data)

r=requests.post(url,data=post_data) #Post
r.encoding="utf-8"
#print(r.text)
sp = BeautifulSoup(r.text,'html.parser')
data = sp.find_all("tr",class_="trip-column")
#station=data[0].find_all("span",class_="location")

print("********"+stations[sstation]+"-"+sstation+" 到 "+stations[estation]+"-"+estation+" 可搭如下:********")
for i in range(0,len(data)):

    


請輸入起始站:高雄
請輸入終點站:屏東
請輸入查詢日期(格式:10/31):10/31
請輸入出發起始時間(格式:09:00):17:00
請輸入出發結束時間(格式:12:00):18:00
********4400-高雄 到 5000-屏東 可搭如下:********
普悠瑪127:出發站-南港 終點站-潮州 出發時間:17:08 抵達時間:17:23 行駛時間:15 分 全票價:$ 48
莒光713:出發站-新左營 終點站-臺東 出發時間:17:13 抵達時間:17:39 行駛時間:26 分 全票價:$ 37
區間3331:出發站-新左營 終點站-枋寮 出發時間:17:27 抵達時間:17:54 行駛時間:27 分 全票價:$ 31
區間3221:出發站-嘉義 終點站-潮州 出發時間:17:39 抵達時間:18:06 行駛時間:27 分 全票價:$ 31
自強125:出發站-七堵 終點站-屏東 出發時間:17:56 抵達時間:18:19 行駛時間:23 分 全票價:$ 48
